In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

# making all pandas columns visable with display command
pd.set_option('display.max_columns', 500)

In [42]:
# df_EPL = pd.read_csv('datasets/EPL.csv', index_col = 0)
df_EPL = pd.read_csv('datasets/EPL_sort.csv', index_col = 0)
df_EPL = df_EPL.reset_index()
df_EPL

,index,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
0,2,1731,2008/2009,1,2008-08-16 00:00:00,489044,8472,8650,0,1,5.50,3.60,1.67,0,0,2,0,13,12,4,11,3,5,15,19,1,8,46,54,0,0,H
1,6,1735,2008/2009,1,2008-08-16 00:00:00,489048,8549,8586,2,1,3.20,3.40,2.25,1,0,2,0,11,11,7,2,5,5,25,27,7,9,53,47,0,0,H
2,7,1736,2008/2009,1,2008-08-16 00:00:00,489049,8559,10194,3,1,1.83,3.50,4.50,1,0,2,0,14,15,5,1,7,6,22,29,4,3,47,53,0,0,A
3,3,1732,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,1,1.91,3.40,4.20,2,0,1,0,14,13,5,7,7,15,15,27,6,10,52,48,0,0,H
4,8,1737,2008/2009,1,2008-08-16 00:00:00,489050,8667,9879,2,1,2.60,3.20,2.80,3,0,0,0,10,9,5,4,5,6,16,14,5,6,53,47,0,0,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,2970,4699,2015/2016,38,2016-05-15 00:00:00,1987597,9825,10252,4,0,1.17,9.00,17.00,0,0,1,0,10,6,7,4,8,0,16,6,5,4,60,40,48,62,H
3036,2977,4706,2015/2016,38,2016-05-15 00:00:00,1987604,10003,8456,1,1,6.00,4.75,1.53,1,0,3,0,5,19,1,5,2,12,14,15,4,4,50,50,46,65,D
3037,2976,4705,2015/2016,38,2016-05-15 00:00:00,1987603,10194,8654,2,1,3.50,3.60,2.15,3,0,1,0,8,12,3,10,2,7,13,18,4,10,45,55,44,38,D
3038,2974,4703,2015/2016,38,2016-05-15 00:00:00,1987601,10261,8586,5,1,4.50,4.00,1.80,1,1,2,0,7,6,11,8,4,8,12,14,3,8,39,61,42,59,D


In [3]:
df_EPL.sort_values('date',ascending=False)

,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
2973,4702,2015/2016,38,2016-05-17 00:00:00,1987600,10260,8678,3,1,1.67,4.20,5.25,1,0,0,0,9,8,5,5,4,2,16,5,7,1,68,32,63,42,H
2978,4707,2015/2016,38,2016-05-15 00:00:00,1987605,9817,8472,2,2,2.05,3.75,3.70,1,0,0,0,10,9,6,2,10,2,9,12,7,4,49,51,60,42,H
2975,4704,2015/2016,38,2016-05-15 00:00:00,1987602,8466,9826,4,1,1.36,5.50,9.00,1,0,2,0,6,14,4,5,13,3,20,15,2,5,49,51,49,80,D
2979,4708,2015/2016,38,2016-05-15 00:00:00,1987606,8659,8650,1,1,2.40,3.60,3.00,3,0,1,0,12,13,5,3,8,3,12,14,7,5,33,67,44,34,H
2972,4701,2015/2016,38,2016-05-15 00:00:00,1987599,8668,9850,3,0,1.80,4.00,4.50,0,0,0,0,6,10,9,4,4,5,17,17,4,11,47,53,34,70,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1732,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,1,1.91,3.40,4.20,2,0,1,0,14,13,5,7,7,15,15,27,6,10,52,48,0,0,H
6,1735,2008/2009,1,2008-08-16 00:00:00,489048,8549,8586,2,1,3.20,3.40,2.25,1,0,2,0,11,11,7,2,5,5,25,27,7,9,53,47,0,0,H
5,1734,2008/2009,1,2008-08-16 00:00:00,489047,8668,8655,2,3,2.00,3.30,4.00,2,0,2,0,11,11,2,8,7,8,14,21,3,4,51,49,0,0,A
1,1730,2008/2009,1,2008-08-16 00:00:00,489043,9825,8659,1,0,1.20,6.50,15.00,0,0,0,0,11,9,12,2,13,3,21,7,7,5,66,34,0,0,H


In [64]:
#current_match_id = 1987598
#current_match = df_EPL[df_EPL['match_api_id'] == current_match_id]
current_match = df_EPL.loc[100]
current_match

index                                27
id                                 1756
season                        2008/2009
stage                                11
date                2008-11-01 00:00:00
match_api_id                     489149
home_team_api_id                   8462
away_team_api_id                   8528
home_team_goal                        1
away_team_goal                        2
B365H                              1.91
B365D                               3.4
B365A                               4.2
home_y_card                           0
home_r_card                           0
away_y_card                           3
away_r_card                           0
home_foulcommit                      11
away_foulcommit                      12
home_shoton                           8
away_shoton                           9
home_shotoff                         10
away_shotoff                          5
home_cross                           22
away_cross                           22


In [65]:
team_away = current_match['home_team_api_id']
cm_date = current_match['date']
print(team_away)
print(cm_date)
# 10 chelsea's game before the match
def get_n_last_game(n, team_name, curr_date):
    n = 10
    df = df_EPL
    df = df[df['date'] < curr_date]
    df_away = df[df['away_team_api_id'] == team_name]
    df_home = df[df['home_team_api_id'] == team_name]
    df = pd.concat([df_away,df_home])
    return df.sort_values('date',ascending = False)[:n]

get_n_last_game(10,team_away,cm_date)

8462
2008-11-01 00:00:00


,index,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
92,16,1745,2008/2009,10,2008-10-29 00:00:00,489138,8650,8462,1,0,1.33,5.00,10.00,0,0,3,0,12,13,9,1,11,3,21,6,7,1,54,46,8,9,A
85,377,2106,2008/2009,9,2008-10-26 00:00:00,489129,8462,9879,1,1,1.83,3.50,4.50,0,0,0,0,8,8,11,5,11,4,19,18,8,6,54,46,13,8,D
74,363,2092,2008/2009,8,2008-10-18 00:00:00,489115,10252,8462,0,0,1.83,3.25,5.00,1,0,4,0,19,19,6,3,8,5,28,6,11,2,58,42,14,10,H
65,357,2086,2008/2009,7,2008-10-05 00:00:00,489109,8462,10194,2,1,1.67,3.40,6.00,0,0,3,0,7,15,11,3,8,4,14,25,2,4,57,43,9,14,A
57,347,2076,2008/2009,6,2008-09-28 00:00:00,489099,8462,8586,2,0,2.50,3.25,3.00,5,0,3,0,17,12,5,7,2,6,12,24,3,6,58,42,6,2,H
45,334,2063,2008/2009,5,2008-09-21 00:00:00,489086,8456,8462,6,0,2.00,3.30,4.00,0,0,1,0,13,15,5,2,4,8,23,13,8,5,59,41,7,6,D
30,327,2056,2008/2009,4,2008-09-13 00:00:00,489079,8462,8549,2,1,2.00,3.50,3.75,2,0,1,0,8,5,12,1,10,6,30,8,3,2,57,43,6,4,H
24,225,1954,2008/2009,3,2008-08-30 00:00:00,489067,8668,8462,0,3,2.10,3.30,3.75,1,0,0,0,4,13,7,2,10,2,24,9,5,3,45,55,3,3,H
19,117,1846,2008/2009,2,2008-08-25 00:00:00,489059,8462,10260,0,1,4.80,3.40,1.80,2,0,2,0,11,13,4,7,8,5,17,13,4,3,36,64,0,1,A
7,9,1738,2008/2009,1,2008-08-17 00:00:00,489051,8455,8462,4,0,1.33,5.00,10.00,0,0,1,0,11,8,6,7,10,5,25,9,8,3,53,47,0,0,H


In [67]:
# get n last game both team (chelsea and lC)
def get_last_n_matches(current_match, n):
    team_away = current_match['home_team_api_id']
    team_home = current_match['away_team_api_id']
    cm_date = current_match['date']
    df_away = get_n_last_game(n,team_away,cm_date)
    df_home = get_n_last_game(n,team_home,cm_date)
    df = pd.concat([df_home,df_away])
    return df

get_last_n_matches(current_match, 10)

,index,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
95,19,1748,2008/2009,10,2008-10-29 00:00:00,489141,9879,8528,2,0,2.38,3.30,3.00,0,0,2,0,10,14,1,4,8,4,12,21,2,2,50,50,23,14,H
86,378,2107,2008/2009,9,2008-10-26 00:00:00,489130,8528,10252,0,4,2.60,3.30,2.75,1,0,2,0,15,8,6,4,11,2,25,15,9,4,46,54,2,8,H
71,362,2091,2008/2009,8,2008-10-18 00:00:00,489114,8650,8528,3,2,1.40,4.33,9.00,1,0,2,0,10,15,8,3,9,3,28,8,4,2,60,40,7,10,D
60,358,2087,2008/2009,7,2008-10-04 00:00:00,489110,8528,8549,0,1,2.10,3.30,3.60,3,0,0,0,11,12,11,2,12,4,26,16,8,2,55,45,7,12,D
58,349,2078,2008/2009,6,2008-09-28 00:00:00,489101,8528,8456,2,1,3.00,3.30,2.38,3,0,4,0,12,21,4,8,6,5,12,11,5,3,51,49,5,9,H
46,332,2061,2008/2009,5,2008-09-21 00:00:00,489084,8586,8528,0,0,1.73,3.60,5.00,2,0,2,0,17,12,7,5,6,9,17,18,3,4,51,49,6,6,H
31,329,2058,2008/2009,4,2008-09-13 00:00:00,489081,8528,8472,1,1,2.05,3.30,4.00,2,0,1,0,10,8,9,3,9,3,36,13,8,6,57,43,7,4,H
23,228,1957,2008/2009,3,2008-08-30 00:00:00,489070,8667,8528,0,5,2.50,3.25,2.88,1,0,0,0,20,13,4,1,7,4,23,12,7,4,48,52,4,0,A
18,119,1848,2008/2009,2,2008-08-24 00:00:00,489061,8528,8455,0,1,8.50,4.50,1.40,1,0,2,0,14,16,9,4,2,3,20,10,7,7,50,50,0,3,A
3,3,1732,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,1,1.91,3.40,4.20,2,0,1,0,14,13,5,7,7,15,15,27,6,10,52,48,0,0,H


In [101]:
#create X and y
from sklearn.preprocessing import OneHotEncoder
X = np.zeros((df_EPL.shape[0], 20, 32))
counter = 0
for i in range(df_EPL.shape[0]):
    current_match = df_EPL.loc[i]
        #print(current_match)
    
    tmp = get_last_n_matches(current_match, 10)
    
    #one hot encode result
    tmp_2 = pd.get_dummies(tmp['Result'], prefix= 'Result')
    tmp = pd.concat([tmp, tmp_2], axis=1)
    tmp = tmp.drop(columns= ['season','date','Result'])
    
    #if shape not 20 then not enough history, it will delete later
    if tmp.shape == (20,32):
        X[i] = tmp
    else:
        counter += 1
X = X[counter:]

print(X.shape)
#print(df_EPL.shape)

(2792, 20, 32)


In [102]:
y = pd.get_dummies(df_EPL['Result'], prefix= 'Result')
y = y[counter:]
print(y.shape)


(2792, 3)


In [103]:
y

,Result_A,Result_D,Result_H
248,0,0,1
249,0,1,0
250,1,0,0
251,0,0,1
252,0,0,1
...,...,...,...
3035,0,0,1
3036,0,1,0
3037,0,1,0
3038,0,1,0


In [ ]:
#print(X,y)

In [ ]:
#X = df_EPL.drop(columns = ['Result','season','date','home_team_api_id','away_team_api_id'])
#y_cat = pd.Categorical(df_EPL['Result'])
#y = y.codes

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [105]:
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (2094, 20, 32)
Train labels shape:  (2094, 3)
Test data shape:  (698, 20, 32)
Test labels shape:  (698, 3)


In [107]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(20,32))
# h = Conv2D(32, (7, 7), strides=(2, 2))(x)
# h = Activation('relu')(x)
# h = Flatten()(h)
h = Dense(16)(x)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 32)]          0         
_________________________________________________________________
dense (Dense)                (None, 20, 16)            528       
_________________________________________________________________
activation (Activation)      (None, 20, 16)            0         
Total params: 528
Trainable params: 528
Non-trainable params: 0
_________________________________________________________________


In [108]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.04)

Epoch 1/5


ValueError: in user code:

    c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:543 train_step  **
        self.compiled_metri